In [10]:
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas  as pd
import pickle
import urllib.request
import urllib.parse
from dateutil.parser import parse
import matplotlib.pyplot as plt
from selenium import webdriver

def codeClear(bill_codes):
    for i,ele in enumerate(bill_codes):
        bill_code = ele.find('a')['href']
        m = re.search("javascript:fGoDetail\('(.*?)', 'billSimpleSearch'\)", bill_code)
        bill_code = m.group(1)
        bill_codes[i] = bill_code
    return bill_codes 

In [8]:
# 0. 환경설정 및 세팅
try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

plt.rcParams["font.family"] = 'NanumGothic'

PATH = r"C:\Users\NH\Documents\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver=webdriver.Chrome(PATH)

C:\Users\NH\AppData\Local\Temp\ipykernel_21112\3687071353.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [17]:
#%% (1) 데이터 수집
# 모든 법률 코드 리스트

# 1단계: 의안법호, 제안일자, 법률코드 크롤링 raw1
driver.get('http://likms.assembly.go.kr/bill/BillSearchResult.do')

In [20]:
driver.find_element(By.XPATH,'//*[@id="si1_label02"]/option[2]').click() #법률안
driver.find_element(By.XPATH,'//*[@id="srchForm"]/div/div[3]/select[1]/option[2]').click() #의원
driver.find_element(By.XPATH,'//*[@id="si1_label03"]/option[4]').click() #공동발의

In [21]:
driver.find_element(By.XPATH,'//*[@id="srchForm"]/div/div[6]/button[1]').click() #공동발의
driver.find_element(By.XPATH,"//select/option[@value='100']").click() #공동발의

In [24]:
bill_num =[]
bill_date = []
bill_codes = []
for i in tqdm(range(1,30)):
    driver.find_element(By.CSS_SELECTOR,"a[href*='javascript:GoPage("+str(i)+")']").click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    bill_codes_sub = soup.findAll("div", {"class": "pl25"})
    bill_num_sub = soup.select('body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr > td:nth-of-type(1)') 
    bill_date_sub = soup.select('body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr > td:nth-of-type(4)') 
    bill_codes.extend(bill_codes_sub)
    bill_num.extend(bill_num_sub)
    bill_date.extend(bill_date_sub)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:42<00:00,  1.46s/it]


In [25]:
# 데이터 클렌징.

for i in range(len(bill_num)):
    bill_num[i] = bill_num[i].text
    bill_date[i] = bill_date[i].text        
    
bill_codes = codeClear(bill_codes)

df = pd.DataFrame({'num':bill_num,'date':bill_date,'code':bill_codes})

In [26]:
df

,num,date,code
0,2124652,2023-09-22,PRC_R2P3O0O9W2W2V1V1U3U0T3O0P9O0O9
1,2124651,2023-09-22,PRC_K2R3Q0R6P1Q9O1W1X4V4V5U6U0T8P9
2,2124650,2023-09-22,PRC_K2J3I0X6Y2W8V1U4C5D6B1A5Y5Z2H1
3,2124649,2023-09-22,PRC_U2V3T0U9S1T9R1N8N1M7N1L3J3K3S7
4,2124648,2023-09-22,PRC_W2W3F0F9D1E9C1B0B4X1Y0W2V0V9T6
...,...,...,...
2895,2121342,2023-04-13,PRC_I2H3H0G4V1U3T1R0A3A0Z2Y6W3F4D7
2896,2121338,2023-04-13,PRC_C2K3L0K4J0R7P1O4M0V1W3U3T1R6O6
2897,2121337,2023-04-13,PRC_H2E3D0A4Y0Y4W1K0F1E1D4J3J6H9Q8
2898,2121336,2023-04-13,PRC_W2V3W0E4D1C3A1B0J3I1G1F7N5O8N1


In [28]:
# save
with open('data/raw1.pickle', 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

In [29]:
# 2단계 의원 리스트 가져오기. raw2
with open('Data/raw1.pickle', 'rb') as f:
    raw1 = pickle.load(f)

members = []

for i in tqdm(raw1.code):    
    with urllib.request.urlopen('http://likms.assembly.go.kr/bill/coactorListPopup.do?billId='+i) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        each_members = soup.select('#periodDiv > div.layerInScroll.coaTxtScroll > div > a')
        each_members = list(map(lambda x: x.text, each_members))                                
        members.append(each_members)

100%|██████████████████████████████████████████████████████████████████████████████| 2900/2900 [03:45<00:00, 12.85it/s]


In [30]:
# 저장. # 오래걸리는 데이터는 원본으로 고유한 이름을 두고 그대로 두자...
with open('data/members.pickle', 'wb') as f:
    pickle.dump(members, f, pickle.HIGHEST_PROTOCOL)
    
members

[['이주환(국민의힘/李周桓)',
  '구자근(국민의힘/具滋根)',
  '김용판(국민의힘/金用判)',
  '김학용(국민의힘/金學容)',
  '김희곤(국민의힘/金熙坤)',
  '백종헌(국민의힘/白宗憲)',
  '안병길(국민의힘/安炳吉)',
  '이인선(국민의힘/李仁善)',
  '이헌승(국민의힘/李憲昇)',
  '전봉민(국민의힘/田奉珉)'],
 ['허은아(국민의힘/許垠娥)',
  '권성동(국민의힘/權性東)',
  '김영식(국민의힘/金英植)',
  '김웅(국민의힘/金雄)',
  '박성중(국민의힘/朴成重)',
  '백종헌(국민의힘/白宗憲)',
  '서일준(국민의힘/徐一俊)',
  '정동만(국민의힘/鄭東萬)',
  '최영희(국민의힘/崔英姬)',
  '하태경(국민의힘/河泰慶)'],
 ['이용빈(더불어민주당/李龍彬)',
  '김병욱(더불어민주당/金炳旭)',
  '문정복(더불어민주당/文貞福)',
  '민병덕(더불어민주당/閔炳德)',
  '이개호(더불어민주당/李介昊)',
  '이병훈(더불어민주당/李炳勳)',
  '이용선(더불어민주당/李庸瑄)',
  '정태호(더불어민주당/鄭泰浩)',
  '조오섭(더불어민주당/曺五燮)',
  '한병도(더불어민주당/韓秉道)'],
 ['맹성규(더불어민주당/孟聖奎)',
  '김민기(더불어민주당/金敏基)',
  '김병욱(더불어민주당/金炳旭)',
  '김승남(더불어민주당/金承南)',
  '민홍철(더불어민주당/閔洪喆)',
  '서동용(더불어민주당/徐東榕)',
  '이소영(더불어민주당/李素永)',
  '장철민(더불어민주당/張喆敏)',
  '최인호(더불어민주당/崔仁昊)',
  '허영(더불어민주당/許榮)',
  '허종식(더불어민주당/許琮植)'],
 ['김미애(국민의힘/金美愛)',
  '권성동(국민의힘/權性東)',
  '김석기(국민의힘/金碩基)',
  '김영식(국민의힘/金英植)',
  '김용판(국민의힘/金用判)',
  '박덕흠(국민의힘/朴德欽)',
  '박수영(국민의힘/朴洙瑩)',
  '백종헌(국민의힘/白宗憲)',
  '이만희(국민의힘/李晩熙)',
  '전주혜(국

In [31]:
# 데이터 클렌징
raw2 = pd.DataFrame({'code':raw1.code,'date':raw1.date,'members':members})


raw2.date =  list(map(lambda x: parse(x), raw2.date))

p = re.compile('[^(/)]+')

for i in tqdm(raw2.members):
        for idxj,j in enumerate(i):
            tmp = p.findall(j)
            i[idxj] = tmp

#저장
with open('Data/raw2.pickle', 'wb') as f:
    pickle.dump(raw2, f, pickle.HIGHEST_PROTOCOL)

100%|███████████████████████████████████████████████████████████████████████████| 2900/2900 [00:00<00:00, 57842.84it/s]


In [32]:
raw2

,code,date,members
0,PRC_R2P3O0O9W2W2V1V1U3U0T3O0P9O0O9,2023-09-22,"[[이주환, 국민의힘, 李周桓], [구자근, 국민의힘, 具滋根], [김용판, 국민의..."
1,PRC_K2R3Q0R6P1Q9O1W1X4V4V5U6U0T8P9,2023-09-22,"[[허은아, 국민의힘, 許垠娥], [권성동, 국민의힘, 權性東], [김영식, 국민의..."
2,PRC_K2J3I0X6Y2W8V1U4C5D6B1A5Y5Z2H1,2023-09-22,"[[이용빈, 더불어민주당, 李龍彬], [김병욱, 더불어민주당, 金炳旭], [문정복,..."
3,PRC_U2V3T0U9S1T9R1N8N1M7N1L3J3K3S7,2023-09-22,"[[맹성규, 더불어민주당, 孟聖奎], [김민기, 더불어민주당, 金敏基], [김병욱,..."
4,PRC_W2W3F0F9D1E9C1B0B4X1Y0W2V0V9T6,2023-09-22,"[[김미애, 국민의힘, 金美愛], [권성동, 국민의힘, 權性東], [김석기, 국민의..."
...,...,...,...
2895,PRC_I2H3H0G4V1U3T1R0A3A0Z2Y6W3F4D7,2023-04-13,"[[이주환, 국민의힘, 李周桓], [구자근, 국민의힘, 具滋根], [권명호, 국민의..."
2896,PRC_C2K3L0K4J0R7P1O4M0V1W3U3T1R6O6,2023-04-13,"[[정필모, 더불어민주당, 鄭必模], [고민정, 더불어민주당, 高旼廷], [기동민,..."
2897,PRC_H2E3D0A4Y0Y4W1K0F1E1D4J3J6H9Q8,2023-04-13,"[[이수진, 더불어민주당, 李壽珍], [강민정, 더불어민주당, 姜旼姃], [강준현,..."
2898,PRC_W2V3W0E4D1C3A1B0J3I1G1F7N5O8N1,2023-04-13,"[[이주환, 국민의힘, 李周桓], [구자근, 국민의힘, 具滋根], [권명호, 국민의..."


In [ ]:

'''
#%% 국회의원 현황 자료. 
status = open("Data\현황.txt", "r")

status = [ x[0:3] for x in status]
status = [ x.replace(' ','') for x in status]

delete_list = ['김경수','김종인','문미옥','박남춘','양승조','자유한','권석창','김종태','박찬우',
               '배덕광','이군현','이완영','이우현','이철우','바른미','송기석','안철수','오세정','최명길',
               '민주평','박준영','정의당','노회찬','윤종오','민중당','대한애','무소속']

for i in delete_list:
    status.remove(i)
    
with open('Data/remove_list.pickle', 'wb') as f:
    pickle.dump(delete_list, f, pickle.HIGHEST_PROTOCOL)
    
#%% 현황 자료 상세.
url = 'http://apis.data.go.kr/9710000/NationalAssemblyInfoService/getMemberCurrStateList?serviceKey=SiQ0MWpF2K6paqhGzxa9lCZTHTetbIDf0FqmxLIhcFq0o47AWRji1dUAGIA69pl34sOq0cbW5T18KE%2F9D9Wzeg%3D%3D&numOfRows=298&pageNo=1'
with urllib.request.urlopen(url) as response:
    html = response.read().decode('utf-8')

soup = BeautifulSoup(html, 'html.parser')


hangle_name = list(map(lambda x: x.text, soup.find_all('empnm')))   
hanja_name = list(map(lambda x: x.text, soup.find_all('hjnm')))   
regin = list(map(lambda x: x.text, soup.find_all('orignm')))   
regin1 =  list(map(lambda x: x[0:2], regin))   
re_elect = list(map(lambda x: x.text, soup.find_all('reelegbnnm')))   

Status = pd.DataFrame({'name':hangle_name, 'hanja':hanja_name,'region':regin1,'region_sep':regin,'re_elect':re_elect})
with open('Data/Status.pickle', 'wb') as f:
    pickle.dump(Status, f, pickle.HIGHEST_PROTOCOL)
    
#%% 고향, 나이, 학교 상세.
    
hangle_name

driver = webdriver.Chrome('C:/Users/dongkeon/Documents/chromedriver')


status2 = []

for i in tqdm(hangle_name):
    driver.get('https://people.search.naver.com/search.naver?sm=tab_hty&where=nexearch&query=&ie=utf8&x=0&y=0')
    driver.find_element_by_xpath("//*[@id='nx_query']").send_keys('')
    driver.find_element_by_xpath("//*[@id='nx_query']").send_keys('국회의원 '+i)
    driver.find_element_by_xpath("//*[@id='search_form']/fieldset/input").click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    count_sub = soup.select('#content > div > h2 > em ')
    driver.find_element_by_xpath("//*[@id='content']/div/div[2]/div[2]/div/div/div/div/a/strong").click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    age_sub = soup.select('div > div.profile_wrap > div.profile_dsc > dl.who > dd.dft > span ')
    birth_sub = soup.select('#content > div > div.profile_wrap > div.profile_dsc > dl.dsc > dd:nth-child(2)')
    school_sub = soup.select('#content > div > div.record_wrap > div:nth-child(4) > dl')
    
    status2.append([i,count_sub,age_sub,birth_sub,school_sub])
                   
    
name = [x[0] for x in status2]               
count = [int(x[1][0].text) for x in status2]     
birth = [x[3][0].text for x in status2]             

age = []
school = []
for i in status2:
    try:
        age_sub = i[2][0].text       
        age.append(age_sub)
    except:
        age.append(None)
    try:
        school_sub = i[4][0].text
        school.append(school_sub)
    except:
        school.append(None)
        
status2 = pd.DataFrame({'name':name,'hanja':hanja_name,'count':count,'age':age,'birth':birth,'school':school,})


## 동명이인 문제 몇개 안되니까 수작업으로 해결!!
    
## 최종저장.
with open('Data/status2.pickle', 'wb') as f:
    pickle.dump(status2, f, pickle.HIGHEST_PROTOCOL)        

'''
    